In [11]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import requests

In [2]:
driver = webdriver.Chrome(executable_path = r'C:\Users\eh063\Downloads\chromedriver_win32\chromedriver.exe')
driver.maximize_window()

# 크롤링 시작

In [54]:
def naver_news_video_crawling(word):
    driver.get('https://search.naver.com/search.naver?where=video&sort=playcount&listmode=v&stype=&playtime=&period=&selected_cp=2%3B1016616097&ninth_cp=1016616097%3A5%&네이버 뉴스&query='+word+'&sm=tab_opt&ie=utf8&nso=so%3Ar%2Cp%3Aall&x_video=')
    
    cnt = 1
    
    # 빈 리스트 생성
    titles = []    # 제목
    urls = []    # url
    dates = []    # 동영상 업로드 날짜
    video_url = []    # 동영상 url
    
    while True:
        try:
            # 페이지 소스 얻어오기
            html = driver.page_source

            # soup에 넣어주기
            soup = BeautifulSoup(html, 'lxml')
        
            # class명이 info_title tit인 항목 모두 가져오기
            class_info_title_tit = soup.find_all('dt', {'class' : 'info_title tit'}) 
            for a in class_info_title_tit:
                class_info_title_tit_a = a.find('a')
    
                # 제목 크롤링
                title = class_info_title_tit_a.get('title')
                titles.append(title)
    
                # 뉴스 url 크롤링
                url = class_info_title_tit_a.get('href')
                urls.append(url)

            # 날짜 크롤링
            test = soup.find_all('span', {'class' : 'info'})
            i = 1
            while i<len(test):
                date = test[i].text
                dates.append(date)
                i = i + 3
            
            # 다음 페이지 넘어가기 : cnt가 1일 때와 이상일 때로 나눠서 구분
            if (cnt == 1):
                driver.find_element_by_xpath('//*[@id="elVideoReportItems"]/div[3]/a[10]').click()
            else:
                driver.find_element_by_xpath('//*[@id="elVideoReportItems"]/div[3]/a[11]').click()
            
            cnt = cnt + 1
        
        except:
            break
    
    # 크롤링한 날짜
    today = datetime.today().strftime("%Y/%m/%d %H:%M:%S")
    
    # 조회수 rank
    rank = [num for num in range(1, len(titles)+1)]
    
    # 데이터프레임 만들기
    df = pd.DataFrame({'title' : titles, 'date' : dates, 'url' : urls, 'today' : today, 'search_word' : word, 'rank' : rank})
    
    # csv 저장
    df.to_csv(word+'.csv', encoding = 'utf-8-sig', index = False)

In [65]:
naver_news_video_crawling('우한 폐렴')

In [55]:
naver_news_video_crawling('신종 코로나 바이러스')

In [60]:
naver_news_video_crawling('코로나19')

In [66]:
woohan = pd.read_csv('우한 폐렴.csv')
woohan

,title,date,url,today,search_word,rank
0,'중국 폐렴' 일본서도 확인…다음 주 '춘절 대이동' 비상,2020.01.16.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,1
1,확진자 접촉한 35명 집중관찰…의심환자 3명은 '음성',2020.01.21.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,2
2,'우한 폐렴' 국내 확진자 4명으로…위기경보 '경계' 격상,2020.01.27.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,3
3,지역구민 초청한 이낙연…'이낙연 동네' 누빈 황교안,4일 전,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,4
4,"""명절 맞아 해외여행 설레요""…공항 북새통",2020.01.23.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,5
...,...,...,...,...,...,...
1003,국내 확진자 이틀째 0명…중국에선 사망-확진 '폭증',2020.02.13.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,1004
1004,"[인터뷰] 새 감염경로, 길어진 잠복기…신종 코로나 '변수'",2020.02.10.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,1005
1005,공항 검역도 선별 진료소도 못 거른 '확진'…이유 있었다,2020.02.10.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,1006
1006,"국내 확진자 총 27명…발병 기준, 어떻게 판단해야?",2020.02.10.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,1007


In [63]:
corona_virus = pd.read_csv('신종 코로나 바이러스.csv')
corona_virus

,title,date,url,today,search_word,rank
0,'우한 폐렴' 2번째 사망자…'사람 간 감염' 우려 커져,2020.01.17.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,1
1,커지는 우한 폐렴 공포…국제비상사태 선포 '초읽기',2020.01.23.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,2
2,베이징도 '유령도시'…감염 위험성에 외출하는 시민 없어,2020.02.04.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,3
3,미국서도 '우한폐렴' 첫 환자 발생…제2의 사스 우려,2020.01.22.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,4
4,정치권도 신종코로나 '비상'…복지위 긴급회의,2020.01.30.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,5
...,...,...,...,...,...,...
1003,국내 이틀째 추가 확진 없어…의심환자 28명 검사 중,2020.01.29.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,1004
1004,줄어드는 중국 행 항공편…아시아나도 일부 노선 중단,2020.01.29.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,1005
1005,"美, 신종코로나 확산에 ""중국, 더 투명해야""…뉴욕증시 진정",2020.01.29.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,1006
1006,일본서 우한 방문 이력 없는 감염자 첫 발생,2020.01.29.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:38:40,신종 코로나 바이러스,1007


In [62]:
corona19 = pd.read_csv('코로나19.csv')
corona19

,title,date,url,today,search_word,rank
0,"[라이브썰전] 김영우 ""중국발 입국자 제한 거론했어야"" vs 김종배 ""신중 접근할 문제""",2일 전,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1
1,청도대남병원서 추가 확진자…정신질환자 91명 검사,1일 전,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,2
2,"문 대통령 위기 경보, 최고 단계인 '심각'으로 격상",32분 전,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,3
3,[정치五감] 코로나19에 텅 빈 대구 마트 진열대 外,2일 전,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,4
4,"中 코로나19 사망자 1,868명...확진 증가세 '감소'",4일 전,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,5
...,...,...,...,...,...,...
1003,'초진 늦은' 광주 환자 접촉자 306명…불안 확산,2020.02.05.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1004
1004,'초진 늦은' 광주 환자 접촉자 306명…불안 확산,2020.02.05.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1005
1005,2명 추가 확진…싱가포르 방문자·16번 환자 딸,2020.02.05.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1006
1006,국내 2명 추가 확진…2번 환자 첫 완치·퇴원,2020.02.05.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1007


# 세가지 데이터프레임 합치기

In [237]:
df = pd.concat([woohan, corona_virus, corona19], ignore_index=True)

In [238]:
df.to_csv('코로나19 관련 네이버뉴스 동영상.csv', index=False)

# 복사본

In [239]:
corona = pd.read_csv('코로나19 관련 네이버뉴스 동영상.csv')
cp_corona = corona.copy()
cp_corona

,title,date,url,today,search_word,rank
0,'중국 폐렴' 일본서도 확인…다음 주 '춘절 대이동' 비상,2020.01.16.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,1
1,확진자 접촉한 35명 집중관찰…의심환자 3명은 '음성',2020.01.21.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,2
2,'우한 폐렴' 국내 확진자 4명으로…위기경보 '경계' 격상,2020.01.27.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,3
3,지역구민 초청한 이낙연…'이낙연 동네' 누빈 황교안,4일 전,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,4
4,"""명절 맞아 해외여행 설레요""…공항 북새통",2020.01.23.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,5
...,...,...,...,...,...,...
3019,'초진 늦은' 광주 환자 접촉자 306명…불안 확산,2020.02.05.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1004
3020,'초진 늦은' 광주 환자 접촉자 306명…불안 확산,2020.02.05.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1005
3021,2명 추가 확진…싱가포르 방문자·16번 환자 딸,2020.02.05.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1006
3022,국내 2명 추가 확진…2번 환자 첫 완치·퇴원,2020.02.05.,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,1007


# date 컬럼 전처리

In [240]:
import re
from datetime import timedelta

In [241]:
p = re.compile('[0-9]+')

for i in range(0, len(cp_corona['date'])):
    if '전' in cp_corona['date'][i]:    # ~시간 전, ~일 전, ~분 전인 행들 추출
        if '일' in cp_corona['date'][i]:    # ~일 전일 때
            time = p.findall(cp_corona['date'][i])    # 정규표현식을 활용하여 숫자만 가져오기
            time = int(time[0])    # 리스트형이므로 첫번째 인덱스만 가져오고, 문자열이기 때문에 정수형으로 변환
            
            today = cp_corona['today'][i]    # today = 크롤링한 시간
            today = datetime.strptime(today, '%Y/%m/%d %H:%M:%S')    # today열은 문자열이므로 날짜형으로 변경
            cp_corona['date'][i] = today - timedelta(days=time)    # date열 = today열 - time   
        elif '시간' in cp_corona['date'][i]:    # ~시간 전일 때
            time = p.findall(cp_corona['date'][i])    # 정규표현식을 활용하여 숫자만 가져오기
            time = int(time[0])    # 리스트형이므로 첫번째 인덱스만 가져오고, 문자열이기 때문에 정수형으로 변환
            
            today = cp_corona['today'][i]    # today = 크롤링한 시간
            today = datetime.strptime(today, '%Y/%m/%d %H:%M:%S')    # today열은 문자열이므로 날짜형으로 변경
            cp_corona['date'][i] = today - timedelta(hours=time)    # date열 = today열 - time  
        else:    # ~분 전일 때
            time = p.findall(cp_corona['date'][i])    # 정규표현식을 활용하여 숫자만 가져오기
            time = int(time[0])    # 리스트형이므로 첫번째 인덱스만 가져오고, 문자열이기 때문에 정수형으로 변환
            
            today = cp_corona['today'][i]    # today = 크롤링한 시간
            today = datetime.strptime(today, '%Y/%m/%d %H:%M:%S')    # today열은 문자열이므로 날짜형으로 변경
            cp_corona['date'][i] = today - timedelta(minutes=time)    # date열 = today열 - time  
    else:
        cp_corona['date'][i] =  datetime.strptime(cp_corona['date'][i], '%Y.%m.%d.')
        cp_corona['today'][i] =  datetime.strptime(cp_corona['today'][i], '%Y/%m/%d %H:%M:%S')
    
    # 년, 월, 일만 남기기
    cp_corona['date'][i] = str(cp_corona['date'][i])[0:10]

C:\Users\eh063\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\eh063\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\eh063\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\eh063\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value i

In [242]:
# 2015년도 데이터가 있어서 삭제
cp_corona = cp_corona[cp_corona.date >= '2019-12-31']

# url 컬럼 전처리  
  
  url 중복 제거

In [243]:
cp_corona = cp_corona.drop_duplicates(['url'])

In [244]:
cp_corona

,title,date,url,today,search_word,rank
0,'중국 폐렴' 일본서도 확인…다음 주 '춘절 대이동' 비상,2020-01-16,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,1
1,확진자 접촉한 35명 집중관찰…의심환자 3명은 '음성',2020-01-21,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,2
2,'우한 폐렴' 국내 확진자 4명으로…위기경보 '경계' 격상,2020-01-27,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,3
3,지역구민 초청한 이낙연…'이낙연 동네' 누빈 황교안,2020-02-19,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,4
4,"""명절 맞아 해외여행 설레요""…공항 북새통",2020-01-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,5
...,...,...,...,...,...,...
3007,'초진 늦은' 광주 환자 접촉자 306명…불안 확산,2020-02-05,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:41:34,코로나19,992
3017,16번 환자 접촉자만 306명…병원발 감염 확산 우려,2020-02-05,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:41:34,코로나19,1002
3021,2명 추가 확진…싱가포르 방문자·16번 환자 딸,2020-02-05,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:41:34,코로나19,1006
3022,국내 2명 추가 확진…2번 환자 첫 완치·퇴원,2020-02-05,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:41:34,코로나19,1007


In [245]:
# date, rank 컬럼으로 정렬
cp_corona = cp_corona.sort_values(by=['date', 'rank'], axis=0).reset_index()

In [246]:
cp_corona = cp_corona.drop('index', axis=1)

In [247]:
cp_corona

,title,date,url,today,search_word,rank
0,중국서 원인불명 폐렴 잇따라…'사스' 우려 확산,2019-12-31,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,809
1,中 우한서 폐렴환자 집단 발생…입국자 검역 강화,2020-01-03,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,779
2,사스 우려 '원인불명 폐렴' 속출 中시장 휴업,2020-01-03,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,780
3,中원인 불명 폐렴 잇따라…중화권 다시 '사스' 공포,2020-01-03,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,805
4,中원인 불명 폐렴 잇따라…중화권 다시 '사스' 공포,2020-01-03,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,807
...,...,...,...,...,...,...
1270,[현장연결] 코로나19 123명 추가 확진…보건당국 브리핑,2020-02-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,241
1271,"[현장연결] 문 대통령 ""정부 위기경보 최고단계 '심각' 상향""",2020-02-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,650
1272,[속보] 코로나19 신규환자 46명…국내 확진자 총 602명,2020-02-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,953
1273,대구서도 첫 사망자 발생…경북대병원 50대 여성,2020-02-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,954


In [253]:
cp_corona.to_csv('[최종]코로나19 관련 네이버뉴스 동영상.csv', encoding = 'utf-8-sig', index=False)

In [254]:
cp_corona = pd.read_csv('[최종]코로나19 관련 네이버뉴스 동영상.csv')
cp_corona

,title,date,url,today,search_word,rank
0,중국서 원인불명 폐렴 잇따라…'사스' 우려 확산,2019-12-31,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,809
1,中 우한서 폐렴환자 집단 발생…입국자 검역 강화,2020-01-03,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,779
2,사스 우려 '원인불명 폐렴' 속출 中시장 휴업,2020-01-03,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,780
3,中원인 불명 폐렴 잇따라…중화권 다시 '사스' 공포,2020-01-03,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,805
4,中원인 불명 폐렴 잇따라…중화권 다시 '사스' 공포,2020-01-03,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020-02-23 17:54:41,우한 폐렴,807
...,...,...,...,...,...,...
1270,[현장연결] 코로나19 123명 추가 확진…보건당국 브리핑,2020-02-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:54:41,우한 폐렴,241
1271,"[현장연결] 문 대통령 ""정부 위기경보 최고단계 '심각' 상향""",2020-02-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,650
1272,[속보] 코로나19 신규환자 46명…국내 확진자 총 602명,2020-02-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,953
1273,대구서도 첫 사망자 발생…경북대병원 50대 여성,2020-02-23,http://news.naver.com/main/read.nhn?mode=LPOD&...,2020/02/23 17:41:34,코로나19,954
